<div style="background-color: #ccffcc; padding: 10px;">
    <h1> Tutorial 3 </h1> 
    <h2> Random Forests </h2>
</div>    

# Overview

<div style="background-color: #ccffcc; padding: 10px;">

<h1>Random Forests </h1>


 
## The very basics
   
    
## Recommended reading 


</div>


<hr>


<div style="background-color: #e6ccff; padding: 10px;">
    
<h1> Machine Learning Theory </h1>
<a href="">
<img src="">
</a>

    
## Random Forests

  
    
</div>    

<div style="background-color: #cce5ff; padding: 10px;">

<h1> Python </h1>

Basic python knowledge is assumed for this tutorial. 
 
    
## Keras
    
</div>
    
<hr>

<div style="background-color: #ffffcc; padding: 10px;">
    
<h1> Requirements </h1>

These notebooks should run with the following requirements satisfied

<h2> Python Packages: </h2>

* Python 3
* tensorflow


<h2> Data Requirements</h2>
    
This notebook referes to some data included in the git hub repositroy
    
</div>


**Contents:**


1. [Leaf Data](#Leaf-Data)
2. [Decision Trees](#Decision-Trees)
3. [Random Forests](#Random-Forests)
4. [K fold cross validation](#K-fold-cross-validation)
4. [HyperParameters](#HyperParameters)
5. [Train Random Forest](#Train-Random-Forest)
6. [Results](#Results) 


<div style="background-color: #cce5ff; padding: 10px;">
Load in all required modules (includig some auxillary code) and turn off warnings. Make sure Keras session is clear
</div>

In [2]:
# For readability: disable warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve

# Leaf Data

<div style="background-color: #ccffcc; padding: 10px;">

Observational data 805 obs of bean leaves multispec device measures

| Feature |
| ------- |
| air temp |
|IR temp leaf temp |
|RH |
| Photosyntheticakky activate radtion (usable light)|
| Photo efficency |
| leaf health (greeness relative chorophyll) |
| proton conductivity |
| leaf thickness |
| leaf angle |   

These feauture are chosen from scientific theory 
</div>

In [3]:
# Set random seed to ensure "reproducible" runs
RSEED = 50
# Load in Data
X = np.array([[2, 2], 
              [2, 1],
              [2, 3], 
              [1, 2], 
              [1, 1],
              [3, 3]])
# Dummy array e.g. precip
y = np.array([0, 1, 1, 1, 0, 1])
# Clean Data
# Plot data?
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18
plt.figure(figsize = (8, 8))

# Plot each point as the label
for x1, x2, label in zip(X[:, 0], X[:, 1], y):
    plt.text(x1, x2, str(label), fontsize = 40, color = 'g',
             ha='center', va='center')
    
# Plot formatting
plt.grid(None);
plt.xlim((0, 3.5));
plt.ylim((0, 3.5));
plt.xlabel('leaf temp', size = 20); plt.ylabel('air temp', size = 20); plt.title('Precip', size = 24)

ModuleNotFoundError: No module named 'pandas'

# Decision Trees

<div style="background-color: #e6ccff; padding: 10px;">

</div>

<div style="background-color: #ccffcc; padding: 10px;">
Leaf stuff...

</div>

In [ ]:
# Make a decision tree and train
tree = DecisionTreeClassifier(random_state=RSEED)
tree.fit(X, y)

<div style="background-color: #cce5ff; padding: 10px;">

<h1> Visualizing and </h1>

</div>

In [ ]:
print(f'Decision tree has {tree.tree_.node_count} nodes with maximum depth {tree.tree_.max_depth}.')
print(f'Model Accuracy: {tree.score(X, y)}')

In [ ]:
# Visulalizing the tree

# Export as dot
export_graphviz(tree, 'tree.dot', rounded = True, 
                feature_names = ['sst', 'time'], 
                class_names = ['0', '1'], filled = True)
# Convert to png make the graphziv a png image
call(['dot', '-Tpng', 'tree.dot', '-o', 'example_tree.png', '-Gdpi=400']);
Image('example_tree.png')

At each node the decision tree considers a feature based question reducing the Gini impurity, using the samples with values (no samples in class 1 and class 2 (0,1 here) to predict the class (0 or 1)

### Gini Impurity 

The probability  that a randomly selected sample from a node will be incorrectly classified according to the distribution of samples in a node. At each split the tree tries to pick values that reduce the gini impurity, here we get to 0 for every training point as no limit was set 

bias- variance trade off, the model is "less accurate" but would perform much better on non training data than the overfitted model...

**random forests** - we could also make lots of trees of subsets of the data and take an average

<hr>

# Example with a real dataset

real data downloaded from [Kaggle](https://www.kaggle.com/cdc/behavioral-risk-factor-surveillance-system), of 100,000 induviduals with features for socioeconomic and lifestyle indicators to predict overall health (0, 1) (poor, good)

data is imbalanced so accuracy is not so useful use:

* recall
* precision
* reciever operating charateristic area under curve (ROC AUC)

## Training

* 70% : 30% split training testing 
* can use sklearn to split the data
* must make sure no missing values (imputation)

In [ ]:

# Extract the labels
labels = np.array(df.pop('label'))

# 30% examples in test data
train, test, train_labels, test_labels = train_test_split(df, labels, 
                                                          stratify = labels,
                                                          test_size = 0.3, 
                                                          random_state = RSEED)
# fill in missing values
train = train.fillna(train.mean())
test = test.fillna(test.mean())

# Features for feature importances
features = list(train.columns)

In [ ]:
# Train tree
tree.fit(train, train_labels)
print(f'Decision tree has {tree.tree_.node_count} nodes with maximum depth {tree.tree_.max_depth}.')

In [ ]:
# Will over fit
# Make probability predictions
train_probs = tree.predict_proba(train)[:, 1]
probs = tree.predict_proba(test)[:, 1]

train_predictions = tree.predict(train)
predictions = tree.predict(test)

In [ ]:


print(f'Train ROC AUC Score: {roc_auc_score(train_labels, train_probs)}')
print(f'Test ROC AUC  Score: {roc_auc_score(test_labels, probs)}')

Visualising predictions vs truth - many false negatives and positives

<hr> 

## feature importance

The good things about these trees is we can detect what the important features were in the decision tree by summoning the Gini Impurity over all the nodes of the tree in which the features are used. e.g here "DIFFWALK" was highly important

In [ ]:
# Save tree as dot file
export_graphviz(tree, 'tree_real_data.dot', rounded = True, 
                feature_names = features, max_depth = 6,
                class_names = ['poor health', 'good health'], filled = True)

# Convert to png
call(['dot', '-Tpng', 'tree_real_data.dot', '-o', 'tree_real_data.png', '-Gdpi=200'])

# Visualize
Image(filename='tree_real_data.png')

In [ ]:
# Save tree as dot file
export_graphviz(tree, 'tree_real_data.dot', rounded = True, 
                feature_names = features, max_depth = 6,
                class_names = ['poor health', 'good health'], filled = True)

# Convert to png
call(['dot', '-Tpng', 'tree_real_data.dot', '-o', 'tree_real_data.png', '-Gdpi=200'])

# Visualize
Image(filename='tree_real_data.png')

## Random Forests

basically an ensemble (1000 or 100,000 s) of decision trees, training each tree on a random set of observations and for each node only a subset of features are used and the predictions are averaged to arrive at he final classifciation

### scikit-learn example

1. 100 trees theat are very deep - each tree will over fit but the forest will not

## K fold cross validation

## Hyper Parameters